In [84]:
import spotipy
import spotipy.util as util
import pickle
import sys
import requests
from tqdm import tqdm_notebook as tqdm
import os
import pandas as pd
import numpy as np
from multiprocessing import Pool
from fuzzywuzzy import fuzz
import string
import json
import unidecode

In [177]:


# Spotify Infos
scope = 'user-library-read'
username = "ilias.miraoui@gmail.com"

token = util.prompt_for_user_token(username, scope, client_id='6d6b00a4a500471e94c5a3f1559e8cc6', client_secret='500950b79db94ac88f6859d24e4e6033', redirect_uri='http://localhost:8888/callback')
if token:
    spotify = spotipy.Spotify(auth=token)
else:
    print("Connexion to spotify API failed.")
    sys.exit()

    


In [69]:
def simplify_string(s):
    return unidecode.unidecode(s.lower()).replace('the', ' ').replace(' ', '').translate(str.maketrans('', '', string.punctuation))


def similar(a, b, threshold=70):
    return fuzz.partial_ratio(simplify_string(a), simplify_string(b)) >= threshold

In [35]:
def get_song_details(artist,track):
    song_details = dict()
    results = spotify.search(q=" track:" + track + ' artist:' + artist  , type='track')
    maxi = None
    if results['tracks']['total'] > 0:
        for i in results['tracks']['items']:
            #and similar(artist, i["artists"][0]["name"], 80)
            if similar(track, i['name'], 80):
                maxi = i
                break
        if maxi is None:
            return None
        if results is None or len(results) == 0:
            return None
        _id = maxi["id"] 
        song_details[_id] = dict()
        song_details[_id]["explicit"] = maxi["explicit"]
        song_details[_id]["duration_ms"] = maxi["duration_ms"]
        song_details[_id]["disc_number"] = maxi["disc_number"]
        song_details[_id]["track_number"] = maxi["track_number"]
        song_details[_id]["album_id"] = maxi["album"]["id"]
        song_details[_id]["album_release_date"] = maxi["album"]["release_date"]
        song_details[_id]["album_release_date_precision"] = maxi["album"]["release_date_precision"]
    return song_details


    

In [5]:
def get_song_features(_id):
    song_features = dict()
    results = spotify.audio_features(_id)
    results = results[0]
    if results != None: 
        song_features["danceability"] = results["danceability"]
        song_features["energy"] = results["energy"]
        song_features["key"] = results["key"]
        song_features["loudness"] = results["loudness"]
        song_features["mode"] = results["mode"]
        song_features["speechiness"] = results["speechiness"]
        song_features["acousticness"] = results["acousticness"]
        song_features["instrumentalness"] = results["instrumentalness"]
        song_features["liveness"] = results["liveness"]
        song_features["valence"] = results["valence"]
        song_features["tempo"] = results["tempo"]
        song_features["time_signature"] = results["time_signature"]
        return song_features
    else:
        return None

In [6]:
def get_label(album_id):
    label = spotify.album(album_id)["label"]
    return label

In [79]:
song = get_song_details("Jimi Hendrix","Purple Haze")
features = get_song_features(song.keys())

In [80]:
song.keys()

dict_keys(['0wJoRiX5K5BxlqZTolB2LD'])

In [8]:
with open('hot-100_data.pkl', 'rb') as f:
        df = pickle.load(f)
with open('hot-100_chart.pkl', 'rb') as f:
        chart = pickle.load(f)

In [215]:
song_df = df.drop_duplicates(["artist","title"])
song_df["spotify_id"] = None

/Users/iliasmiraoui/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [273]:
song_df["spotify_album_id"] = None

/Users/iliasmiraoui/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [168]:
i=0
for song in song_df.index:
    if pd.isna(song_df.loc[song,"spotify_album_id"]) == True:
        title = song_df.loc[song,"title"]
        artist = song_df.loc[song,"artist"]
        song_details = get_song_details(artist,title)
        if type(song_details) == dict and len(song_details)>0 :
            song_df.loc[song,"spotify_id"] = list(song_details.keys())[0]
            for feature in list(song_details[list(song_details.keys())[0]].keys()):
                song_df.loc[song,"spotify_"+feature] = song_details[list(song_details.keys())[0]][feature]
            song_df.loc[song,"album_label"] = get_label(song_df.loc[song,"spotify_album_id"])
        else:
            i = i+1
            print(i)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148


KeyboardInterrupt: 

In [70]:
song_df[pd.isna(song_df["spotify_id"])]

,artist,title,isnew,rank,date,spotify_id,spotify_explicit,spotify_duration_ms,spotify_disc_number,spotify_track_number,...,spotify_acousticness,spotify_instrumentalness,spotify_liveness,spotify_valence,spotify_tempo,spotify_time_signature,spotify_album_id,spotify_album_release_date,spotify_album_release_date_precision,album_label
6,Chris Brown Featuring Drake,No Guidance,False,7,2019-11-16,None,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN
10,Young Thug Featuring Gunna,Hot,False,11,2019-11-16,None,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN
12,Lil Tecca,Ran$om,False,13,2019-11-16,None,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN
14,Ed Sheeran Featuring Khalid,Beautiful People,False,15,2019-11-16,None,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN
17,Lil Nas X Featuring Billy Ray Cyrus,Old Town Road,False,18,2019-11-16,None,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112899,Kim Sanders,Jealousy,False,100,1998-04-11,None,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN
113096,Mic Geronimo Featuring DMX & Black Rob,Nothin' Move But The Money,False,97,1998-03-28,None,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN
113168,Richard Marx & Donna Lewis,At The Beginning,False,69,1998-03-21,None,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN
113172,She Moves,Breaking All The Rules,False,73,1998-03-21,None,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN



 51%|█████     | 4330/8519 [03:50<09:03,  7.70it/s]

In [12]:
with open('song_df_100.pkl', 'rb') as f:
    song_df = pickle.load(f)

In [11]:
song_df["spotify_key"] = None

In [169]:
for song in song_df[song_df["spotify_id"].notnull()].index:
    spotify_key = song_df.loc[song,"spotify_key"]
    spotify_id = song_df.loc[song,"spotify_id"]
    if pd.isna(spotify_key) == True:
        song_features = get_song_features(spotify_id)
        if song_features != None:
            for feature in list(song_features.keys()):
                song_df.loc[song,"spotify_"+feature] = song_features[feature]

In [175]:
song_df

,artist,title,isnew,rank,date,spotify_id,spotify_explicit,spotify_duration_ms,spotify_disc_number,spotify_track_number,...,spotify_acousticness,spotify_instrumentalness,spotify_liveness,spotify_valence,spotify_tempo,spotify_time_signature,spotify_album_id,spotify_album_release_date,spotify_album_release_date_precision,album_label
0,Lewis Capaldi,Someone You Loved,False,1,2019-11-16,7qEHsqek33rTcFNT9PFqLf,False,182160.0,1.0,4.0,...,0.7510,0.00000,0.1050,0.4460,109.891,4.0,5658aM19fA3JVwTK6eQX70,2019-05-17,day,Vertigo Berlin
1,Post Malone,Circles,False,2,2019-11-16,21jGcNKet2qwijlDFuPiPb,False,215280.0,1.0,6.0,...,0.1920,0.00244,0.0863,0.5530,120.042,4.0,4g1ZRSobMefqF6nelkgibi,2019-09-06,day,Republic Records
2,Shawn Mendes & Camila Cabello,Senorita,False,3,2019-11-16,0TK2YIli7K1leLovkQiNik,False,190960.0,1.0,1.0,...,0.0370,0.00000,0.0945,0.7500,116.947,4.0,2ZaX1FdZCwchXl1QZiD4O4,2019-06-21,day,Island Records
3,Lizzo,Truth Hurts,False,4,2019-11-16,5qmq61DAAOUaW8AUo8xKhh,True,173325.0,1.0,13.0,...,0.1100,0.00000,0.1230,0.4120,158.087,4.0,74gSdSHe71q7urGWMMn3qB,2019-05-03,day,Nice Life/Atlantic
4,Selena Gomez,Lose You To Love Me,False,5,2019-11-16,1HfMVBKM75vxSfsQ5VefZ5,False,206458.0,1.0,1.0,...,0.5760,0.00000,0.2100,0.0916,101.993,4.0,3tBkjgxDqAwss76O1YHsSY,2019-10-23,day,Interscope Records
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113178,Spice Girls,Spice Up Your Life,False,79,1998-03-21,5qGwqO0lkbBXw4xNfzT7SF,False,173666.0,1.0,1.0,...,0.0105,0.03410,0.4030,0.7020,124.016,4.0,3sr6lAuO3nmB1u8ZuQgpiX,1997-01-01,day,Virgin Records
113192,Phajja,"So Long (Well, Well, Well)",False,93,1998-03-21,7phS4S15qLjPC7JlFnzWql,False,262333.0,1.0,23.0,...,0.4320,0.00168,0.0446,0.5450,79.913,4.0,6qbvgcaWgMn2BQY6FhdOm2,2018-11-16,day,Warner Music Group - X5 Music Group
113194,Sting & The Police,Roxanne `97 - Puff Daddy Remix,False,95,1998-03-21,0RlWNvJKKp0gODH7sUwm1j,False,275493.0,1.0,7.0,...,0.0579,0.82900,0.0561,0.7230,97.518,4.0,0Av1HIrrBAtIbQTaAqo8Lb,2011-11-07,day,2011 Ameritz Music Ltd
113198,Shania Twain,Don't Be Stupid (You Know I Love You),False,99,1998-03-21,6xAF3xYDK915NEvC6WNbj9,False,214240.0,1.0,4.0,...,0.2610,0.00000,0.5480,0.9030,122.025,4.0,4UMe0Ods7kygK6OISasZe9,1997-11-04,day,Mercury Nashville


In [47]:
artist = "Shawn Mendes"
track = "Senorita"
results = spotify.search(q=" track:" + track + ' artist:' + artist  , type='track')

In [67]:
# Multiple artists issue

def get_song_details_multiple(artist,track):
    song_details = dict()
    artist = artist.split("&")[0].strip() 
    results = spotify.search(q=" track:" + track + ' artist:' + artist  , type='track')
    maxi = None
    if results['tracks']['total'] > 0:
        for i in results['tracks']['items']:
            if similar(track, i['name'], 80):
                maxi = i
                break
        if maxi is None:
            return None
        if results is None or len(results) == 0:
            return None
        _id = maxi["id"] 
        song_details[_id] = dict()
        song_details[_id]["explicit"] = maxi["explicit"]
        song_details[_id]["duration_ms"] = maxi["duration_ms"]
        song_details[_id]["disc_number"] = maxi["disc_number"]
        song_details[_id]["track_number"] = maxi["track_number"]
        song_details[_id]["album_id"] = maxi["album"]["id"]
        song_details[_id]["album_release_date"] = maxi["album"]["release_date"]
        song_details[_id]["album_release_date_precision"] = maxi["album"]["release_date_precision"]
    return song_details

i=0
pbar = tqdm(song_df.index)
for song in pbar:
    if pd.isna(song_df.loc[song,"spotify_id"]) == True:
        title = song_df.loc[song,"title"]
        artist = song_df.loc[song,"artist"]
        song_details = get_song_details_multiple(artist,title)
        if type(song_details) == dict and len(song_details)>0 :
            song_df.loc[song,"spotify_id"] = list(song_details.keys())[0]
            for feature in list(song_details[list(song_details.keys())[0]].keys()):
                song_df.loc[song,"spotify_"+feature] = song_details[list(song_details.keys())[0]][feature]
            song_df.loc[song,"album_label"] = get_label(song_df.loc[song,"spotify_album_id"])
        else:
            i = i+1
            print(i)


  0%|          | 11/8519 [00:00<03:29, 40.70it/s]

1
2


  0%|          | 15/8519 [00:00<06:35, 21.49it/s]

3
4


  0%|          | 21/8519 [00:00<06:21, 22.26it/s]

...

  8%|▊         | 723/8519 [00:28<05:19, 24.43it/s]

199
200


  9%|▊         | 737/8519 [00:28<04:03, 31.95it/s]

201
202


KeyboardInterrupt: 

In [ ]:
# Track name... issue

def get_song_details_simpler(artist,track):
    song_details = dict()
    artist = artist.split(" & ")[0]
    artist = artist.split(" x ")[0]
    artist = artist.split(" + ")[0]
    artist = artist.split(", ")[0]
    artist = artist.split(" From ")[0]
    artist = artist.split("(")[0]
    artist = artist.split(" Presents ")[0]
    artist = artist.split(" featuring ")[0]
    artist = artist.split(" Featuring ")[0]
    artist = artist.split(" Feauring ")[0]
    artist = artist.split(" Or ")[0]
    artist = artist.split(" And ")[0]
    track = track.replace("****","igga")
    track = track.replace("***","igg")
    track = track.replace("**","uc")
    track = track.replace(":",".")
    track = track.replace("$","s")
    
    results = spotify.search(q=" track:" + track + ' artist:' + artist  , type='track')
    maxi = None
    if results['tracks']['total'] > 0:
        for i in results['tracks']['items']:
            spotify_track = i['name'].split("(")[0]
            spotify_track = i['name'].split(" - ")[0]
            print(spotify_track)
            print(track)
            if similar(track,spotify_track, 60):
                maxi = i
                break
        if maxi is None:
            return None
        if results is None or len(results) == 0:
            return None
        _id = maxi["id"] 
        song_details[_id] = dict()
        song_details[_id]["explicit"] = maxi["explicit"]
        song_details[_id]["duration_ms"] = maxi["duration_ms"]
        song_details[_id]["disc_number"] = maxi["disc_number"]
        song_details[_id]["track_number"] = maxi["track_number"]
        song_details[_id]["album_id"] = maxi["album"]["id"]
        song_details[_id]["album_release_date"] = maxi["album"]["release_date"]
        song_details[_id]["album_release_date_precision"] = maxi["album"]["release_date_precision"]
    return song_details

i=0
pbar = tqdm(song_df.index)
for song in pbar:
    if pd.isna(song_df.loc[song,"spotify_id"]) == True:
        title = song_df.loc[song,"title"]
        artist = song_df.loc[song,"artist"]
        song_details = get_song_details_simpler(artist,title)
        if type(song_details) == dict and len(song_details)>0 :
            song_df.loc[song,"spotify_id"] = list(song_details.keys())[0]
            for feature in list(song_details[list(song_details.keys())[0]].keys()):
                song_df.loc[song,"spotify_"+feature] = song_details[list(song_details.keys())[0]][feature]
            song_df.loc[song,"album_label"] = get_label(song_df.loc[song,"spotify_album_id"])
        else:
            i = i+1
            print(i)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119


In [171]:
with open('song_df.pkl', 'wb') as f:
    pickle.dump(song_df, f)

In [81]:
artist = " & Featuring et"
artist = artist.replace("&","")
artist = artist.replace("Featuring","")

In [172]:
song_df[pd.isna(song_df["spotify_id"])]

,artist,title,isnew,rank,date,spotify_id,spotify_explicit,spotify_duration_ms,spotify_disc_number,spotify_track_number,...,spotify_acousticness,spotify_instrumentalness,spotify_liveness,spotify_valence,spotify_tempo,spotify_time_signature,spotify_album_id,spotify_album_release_date,spotify_album_release_date_precision,album_label
9866,Addison Agen,Tennessee Rain,True,67,2018-01-03,None,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN
9868,Chloe Kohanski,Wish I Didn't Love You,True,69,2018-01-03,None,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN
9883,Red Marlow,I Pray,True,84,2018-01-03,None,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN
10198,Chloe Kohanski,Total Eclipse Of The Heart,True,99,2017-12-16,None,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN
10595,"Quavo, Takeoff & Offset",Too Hotty,False,96,2017-11-18,None,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111983,Jay-Z Featuring BLACKstreet,The City Is Mine,False,84,1998-06-13,None,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN
112299,Innerlude,I Don't Wanna Go On,False,100,1998-05-23,None,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN
112694,PRINCE BE & Ky-Mani,Gotta Be...Movin' On Up,False,95,1998-04-25,None,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN
112696,KAI,Say You'll Stay,False,97,1998-04-25,None,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN


In [156]:
artist = "Iggy Azalea"
track = "Kream"
spotify.search(q=track+" "+artist  , type='track')
spotify.search(q=" track:" + track + ' artist:' + artist  , type='track')


{'tracks': {'href': 'https://api.spotify.com/v1/search?query=+track%3AKream+artist%3AIggy+Azalea&type=track&offset=0&limit=10',
  'items': [{'album': {'album_type': 'single',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/5yG7ZAZafVaAlMTeBybKAL'},
       'href': 'https://api.spotify.com/v1/artists/5yG7ZAZafVaAlMTeBybKAL',
       'id': '5yG7ZAZafVaAlMTeBybKAL',
       'name': 'Iggy Azalea',
       'type': 'artist',
       'uri': 'spotify:artist:5yG7ZAZafVaAlMTeBybKAL'}],
     'available_markets': ['AD',
      'AR',
      'AT',
      'AU',
      'BE',
      'BG',
      'BO',
      'BR',
      'CA',
      'CH',
      'CL',
      'CO',
      'CR',
      'CY',
      'CZ',
      'DE',
      'DK',
      'DO',
      'EC',
      'EE',
      'ES',
      'FI',
      'FR',
      'GB',
      'GR',
      'GT',
      'HK',
      'HN',
      'HU',
      'ID',
      'IE',
      'IL',
      'IN',
      'IS',
      'IT',
      'JP',
      'LI',
      'LT',
      'LU',
   